In [11]:
import requests
import json
import pandas as pd

In [71]:
url = "https://content.toto.nl/content-service/api/v1/q/popular-bets-event-list?minPrice=1.25&maxPrice=9.90&combinable=true&numOutcomes=3&startTimeFrom=2024-11-09T13%3A32%3A35Z&startTimeTo=2024-11-10T13%3A32%3A35Z&marketSortsIncluded=HH%2CMR&lang=nl-NL&channel=I"

payload = {}
headers = {
  'accept': 'application/json',
  'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
}

response = requests.request("GET", url, headers=headers, data=payload)

match_data = json.loads(response.text)


In [73]:
# Initialize an empty list to collect match data
matches = []

# Extract match details from each event
for event in match_data['data']['events']:
    match = {
        "event_id": event.get("id"),
        "match_name": event.get("name"),
        "start_time": event.get("startTime"),
        "home_team": next((team['name'] for team in event['teams'] if team['side'] == "HOME"), None),
        "away_team": next((team['name'] for team in event['teams'] if team['side'] == "AWAY"), None),
        "competition": event['type']['name'],
        "country": event['class']['name'],
        "sport": event['category']['name'],
    }
    
    # Extract odds, if available
    outcomes = event.get("markets", [{}])[0].get("outcomes", [])
    if outcomes:
        match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "H"), None)
        match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "A"), None)
    
    matches.append(match)

# Convert the match list to a DataFrame
matches_df = pd.DataFrame(matches)

# Display the resulting DataFrame
matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
0,5537946,FC Twente vs Ajax,2024-11-10T13:30:00Z,FC Twente,Ajax,Nederland Eredivisie,Nederland,Voetbal,NaN,2.44
1,5118913,Galatasaray SK vs Samsunspor,2024-11-10T13:00:00Z,Galatasaray SK,Samsunspor,Turkije Süper Lig,Turkije,Voetbal,1.40,NaN
2,5537948,SC Heerenveen vs Go Ahead Eagles,2024-11-10T13:30:00Z,SC Heerenveen,Go Ahead Eagles,Nederland Eredivisie,Nederland,Voetbal,2.75,NaN


In [74]:
import requests

for event_id in  matches_df['event_id']:
    url = f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?eventIds={event_id}&includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"

    payload = {}
    headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    json_data = json.loads(response.text)

    # Initialize empty lists to store the data
    event_ids = []
    event_names = []
    market_names = []
    outcome_names = []
    odds_decimal = []
    prices_numerator = []
    prices_denominator = []
    outcome_types = []
    outcome_subtypes = []

    # Iterate over the events, markets, and outcomes
    for event in json_data['data']['events']:  # Iterate over each event
        event_id = event['id']
        event_name = event['name']
        
        for market in event['markets']:  # Iterate over each market in the event
            market_name = market['name']
            
            for outcome in market['outcomes']:  # Iterate over outcomes for each market
                outcome_name = outcome['name']
                odds_decimal_value = outcome['prices'][0]['decimal']
                numerator = outcome['prices'][0]['numerator']
                denominator = outcome['prices'][0]['denominator']
                outcome_type = outcome['type']
                outcome_subtype = outcome['subType']
                
                # Append captured values to the respective lists
                event_ids.append(event_id)
                event_names.append(event_name)
                market_names.append(market_name)
                outcome_names.append(outcome_name)
                odds_decimal.append(odds_decimal_value)
                prices_numerator.append(numerator)
                prices_denominator.append(denominator)
                outcome_types.append(outcome_type)
                outcome_subtypes.append(outcome_subtype)

    # Create a DataFrame with the collected data
    df = pd.DataFrame({
        'Event ID': event_ids,
        'Event Name': event_names,
        'Market Name': market_names,
        'Outcome Name': outcome_names,
        'Odds (Decimal)': odds_decimal,
        'Price Numerator': prices_numerator,
        'Price Denominator': prices_denominator,
        'Outcome Type': outcome_types,
        'Outcome SubType': outcome_subtypes
    })

    # Show the DataFrame
    df

In [75]:
# Get all offers for all matches of Football event_ids
# Initialize an empty list to accumulate all data
all_data = []

# Loop over all event_ids in your matches_df
for event_id in matches_df['event_id']:
    url = f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?eventIds={event_id}&includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"

    payload = {}
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

    response = requests.get(url, headers=headers, data=payload)
    json_data = json.loads(response.text)

    # Initialize empty lists to store the data for this event
    event_ids = []
    event_names = []
    market_names = []
    outcome_names = []
    odds_decimal = []
    prices_numerator = []
    prices_denominator = []
    outcome_types = []
    outcome_subtypes = []

    # Iterate over the events, markets, and outcomes
    for event in json_data['data']['events']:  # Iterate over each event
        event_id = event['id']
        event_name = event['name']
        
        for market in event['markets']:  # Iterate over each market in the event
            market_name = market['name']
            
            for outcome in market['outcomes']:  # Iterate over outcomes for each market
                outcome_name = outcome['name']
                odds_decimal_value = outcome['prices'][0]['decimal']
                numerator = outcome['prices'][0]['numerator']
                denominator = outcome['prices'][0]['denominator']
                outcome_type = outcome['type']
                outcome_subtype = outcome['subType']
                
                # Append captured values to the respective lists
                event_ids.append(event_id)
                event_names.append(event_name)
                market_names.append(market_name)
                outcome_names.append(outcome_name)
                odds_decimal.append(odds_decimal_value)
                prices_numerator.append(numerator)
                prices_denominator.append(denominator)
                outcome_types.append(outcome_type)
                outcome_subtypes.append(outcome_subtype)

    # Create a temporary DataFrame for this batch
    temp_df = pd.DataFrame({
        'Event ID': event_ids,
        'Event Name': event_names,
        'Market Name': market_names,
        'Outcome Name': outcome_names,
        'Odds (Decimal)': odds_decimal,
        'Price Numerator': prices_numerator,
        'Price Denominator': prices_denominator,
        'Outcome Type': outcome_types,
        'Outcome SubType': outcome_subtypes
    })
    
    # Append the temporary DataFrame to the all_data list
    all_data.append(temp_df)

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

     Event ID                        Event Name           Market Name  \
0     5537946                 FC Twente vs Ajax           Draw No Bet   
1     5537946                 FC Twente vs Ajax           Draw No Bet   
2     5537946                 FC Twente vs Ajax          Dubbele Kans   
3     5537946                 FC Twente vs Ajax          Dubbele Kans   
4     5537946                 FC Twente vs Ajax          Dubbele Kans   
...       ...                               ...                   ...   
1067  5537948  SC Heerenveen vs Go Ahead Eagles  Team schoten op doel   
1068  5537948  SC Heerenveen vs Go Ahead Eagles  Team schoten op doel   
1069  5537948  SC Heerenveen vs Go Ahead Eagles  Team schoten op doel   
1070  5537948  SC Heerenveen vs Go Ahead Eagles  Team schoten op doel   
1071  5537948  SC Heerenveen vs Go Ahead Eagles  Team schoten op doel   

                                   Outcome Name  Odds (Decimal)  \
0                                          Ajax         

In [69]:
# Get all bets for 1 match

# import pandas as pd

# # Initialize empty lists to store the data
# event_ids = []
# event_names = []
# market_names = []
# outcome_names = []
# odds_decimal = []
# prices_numerator = []
# prices_denominator = []
# outcome_types = []
# outcome_subtypes = []

# # Iterate over the events, markets, and outcomes
# for event in json_data['data']['events']:  # Iterate over each event
#     event_id = event['id']
#     event_name = event['name']
    
#     for market in event['markets']:  # Iterate over each market in the event
#         market_name = market['name']
        
#         for outcome in market['outcomes']:  # Iterate over outcomes for each market
#             outcome_name = outcome['name']
#             odds_decimal_value = outcome['prices'][0]['decimal']
#             numerator = outcome['prices'][0]['numerator']
#             denominator = outcome['prices'][0]['denominator']
#             outcome_type = outcome['type']
#             outcome_subtype = outcome['subType']
            
#             # Append captured values to the respective lists
#             event_ids.append(event_id)
#             event_names.append(event_name)
#             market_names.append(market_name)
#             outcome_names.append(outcome_name)
#             odds_decimal.append(odds_decimal_value)
#             prices_numerator.append(numerator)
#             prices_denominator.append(denominator)
#             outcome_types.append(outcome_type)
#             outcome_subtypes.append(outcome_subtype)

# # Create a DataFrame with the collected data
# df = pd.DataFrame({
#     'Event ID': event_ids,
#     'Event Name': event_names,
#     'Market Name': market_names,
#     'Outcome Name': outcome_names,
#     'Odds (Decimal)': odds_decimal,
#     'Price Numerator': prices_numerator,
#     'Price Denominator': prices_denominator,
#     'Outcome Type': outcome_types,
#     'Outcome SubType': outcome_subtypes
# })

# # Show the DataFrame
# df

    Event ID         Event Name     Market Name  \
0    5537946  FC Twente vs Ajax     Draw No Bet   
1    5537946  FC Twente vs Ajax     Draw No Bet   
2    5537946  FC Twente vs Ajax    Dubbele Kans   
3    5537946  FC Twente vs Ajax    Dubbele Kans   
4    5537946  FC Twente vs Ajax    Dubbele Kans   
..       ...                ...             ...   
451  5537946  FC Twente vs Ajax  Match Specials   
452  5537946  FC Twente vs Ajax  Match Specials   
453  5537946  FC Twente vs Ajax  Match Specials   
454  5537946  FC Twente vs Ajax  Match Specials   
455  5537946  FC Twente vs Ajax  Match Specials   

                            Outcome Name  Odds (Decimal)  Price Numerator  \
0                                   Ajax            1.75                3   
1                              FC Twente            2.08               27   
2                     Gelijkspel of Ajax            1.42               21   
3                      FC Twente of Ajax            1.33               33   
4 